<a href="https://www.kaggle.com/code/sansisalvo/agrichat?scriptVersionId=167442007" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Agricultural Chat Bot

My goal is to help farmers to produce food effectively by using this chat-bot. They can learn everything from this chat-bot.

# **Install dependencies**

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


In [2]:
import os

os.environ["KERAS_BACKEND"] = "torch"  # Or "torch" or "tensorflow".


In [3]:
import torch
import keras_nlp
import keras
print(torch.__version__)
print("KerasNLP version:", keras_nlp.__version__)
print("Keras Version",keras.__version__)

2024-03-17 11:50:00.016981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 11:50:00.017111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 11:50:00.307570: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.1.2
KerasNLP version: 0.8.2
Keras Version 3.0.5


# Dataset

In [4]:
import json
with open("/kaggle/input/combined-dataset/concatenated_output.json","r") as f:
    data = json.load(f)

def format_data(example):
    template = "Question:\n{question}\n\nAnswer:\n{answer}"
    return template.format(question=example['questions'], answer=example['answers'])

formatted_data = [format_data(example) for example in data]
formatted_data = formatted_data[:2845]

print(formatted_data[:2])
print(len(formatted_data))


['Question:\nWhat is the aim of the PM-KISAN Scheme (2023)?\n\nAnswer:\nThe PM-KISAN Scheme aims to provide income support to all landholding farmers across the country, enabling them to meet their agricultural needs and improve their livelihoods.', 'Question:\nHow does the Agriculture Infrastructure Fund (2023) contribute to the agricultural sector?\n\nAnswer:\nLaunched to enhance post-harvest management and promote infrastructure development in the agriculture sector, this policy provides financial support to farmers for building storage facilities and transportation networks.']
2845


# Load the model

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma/keras/gemma_2b_en/2")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

# LoRA (low-rank adaptation of large language models)


In [6]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

# Train the model on custom dataset

In [7]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 128
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(formatted_data, epochs=2, batch_size=1)

Epoch 1/2
2845/2845 ━━━━━━━━━━━━━━━━━━━━ 1271s 446ms/step - loss: 1.0693 - sparse_categorical_accuracy: 0.4717
Epoch 2/2
2845/2845 ━━━━━━━━━━━━━━━━━━━━ 1272s 447ms/step - loss: 0.9135 - sparse_categorical_accuracy: 0.5153


# Inference

In [8]:
template = "Question:\n{question}\nAnswer:\n{answer}"
prompt = template.format(
    question="What should I do to grow potatoes at home?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
What should I do to grow potatoes at home?
Answer:
1. Select seed potatoes that are disease-free and have good eyes.
2. Plant the seed potatoes in a well-draining soil. Space the seed potatoes about 12 inches apart in mounds.
3. Water the soil thoroughly and mulch the mounds with grass clippings or straw to conserve moisture.
4. Keep the soil evenly moist throughout the growing season.
5. Harvest the potatoes when they reach the desired size and maturity.


In [9]:
save_model= "model.keras"
gemma_lm.save(save_model)

# References

* https://arxiv.org/abs/2106.09685
* https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora